# Modulo pysam per formato SAM/BAM

`pysam` è il modulo Python che mette a disposizione le funzionalità per manipolare file in formato SAM/BAM.

Importare il modulo `pysam`

In [35]:
import pysam

## Leggere gli allineamenti

La classe `AlignmentFile` è la classe che rappresenta un SAM/BAM file.

Un oggetto di tipo `AlignmentFile` è ottenuto a partire da un file SAM/BAM nel seguente modo: 

    sam_file = AlignmentFile(sam_file_name, 'r')
    bam_file = AlignmentFile(bam_file_name, 'rb')

Importare la classe `AlignmentFile`.

In [36]:
from pysam import AlignmentFile

### Ottenere gli allineamenti da un file BAM

Indicizzare il file tramite la funzione `index()`.

In [37]:
pysam.index('./sample.bam')

''

Costruire l'oggetto `AlignmentFile`.

In [38]:
bam_file = AlignmentFile('./sample.bam', 'rb')

`bam_file` contiene tutti gli allineamenti presenti nel file.

Il numero di reads mappati e non mappati si ottiene accedendo agli attributi `mapped` e `unmapped`.

Reads non mappati dovuti a:

- reads che non sono stati allineati a una posizione specifica del genoma di riferimento ma si inseriscono lo stesso per poter fare debugging
- paired-end reads in cui una delle due parti è mappata e l'altra no (ma si inserisce lo stesso)

Informazione riportata nel campo FLAG.

In [39]:
bam_file.mapped

25008

In [40]:
bam_file.unmapped

0

Le references coinvolte nel file BAM si ottengono accedendo all'attributo `references`.

In [41]:
bam_file.references

('X',)

Il numero di references coinvolte nel file BAM si ottengono accedendo all'attributo `nreferences`.

In [42]:
bam_file.nreferences

1

La lunghezza delle references coinvolte nel file BAM si ottengono accedendo all'attributo `lengths`.

In [43]:
bam_file.lengths

(23542271,)

La lunghezza di una reference in particolare si ottiene con il metodo `get_reference_length()`.

In [44]:
bam_file.get_reference_length('X')

23542271

Il numero di reads mappati a una determinata regione della reference si ottiene con il metodo `count()`

In [45]:
bam_file.count('X', 280000, 300000)

25008

## Estrarre gli allineamenti

Il metodo `head(n)` restituisce i primi n allineamenti.

Il metodo `fetch()` invocato nei tre seguenti modi: 

    bam_file.fetch()
    bam_file.fetch(ref_name)
    bam_file.fetch(ref_name, start_region, end_region)
    
restituisce un iteratore contenente (rispettivamente):

- tutti gli allineamenti
- tutti gli allineamenti che si riferiscono alla reference `ref_name`
- tutti gli allineamenti che si riferiscono alla sottoregione tra le posizioni `start_region` e `end_region`della reference `ref_name`.

Il singolo allineamento è un oggetto della classe `AlignedSegment`.

---
    
**Esercizio**: estrarre tutti gli allineamenti.

In [46]:
all_alignments = list(bam_file.fetch())

Il numero di allineamenti estratti è:

In [47]:
len(all_alignments)

25008

**Esercizio**: estrarre gli allineamenti relativi alla reference `X` che cadono nella regione tra le posizioni `280000` e `285000`.

In [48]:
len(list(bam_file.fetch('X', 280000, 285000)))

3170

**Esercizio**: estrarre i primi 50 allineamenti.

## Accedere alle informazioni di un allineamento

Estrarre il primo allineamento della lista `all_alignments`.

In [49]:
first_alignment = all_alignments[0]
first_alignment

Gli oggetti `AlignedSegment` hanno una serie di attributi che descrivono l'allineamento.

In [50]:
first_alignment.cigarstring

'100M'

In [51]:
first_alignment.cigartuples

[(0, 100)]

In [52]:
first_alignment.flag

0

In [53]:
first_alignment.is_paired

False

In [54]:
first_alignment.is_reverse

False

In [55]:
first_alignment.is_secondary

False

In [56]:
first_alignment.mapping_quality

255

In [57]:
first_alignment.reference_start

283185

In [58]:
first_alignment.query_name

'FBtr0300326_e_2967_X_283185'

In [59]:
first_alignment.query_qualities

array('B', [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [60]:
first_alignment.query_alignment_sequence

'GCCCTTGCATTTATTTCCAGTTTTGGTTTTTTTTTTGTATTTATCAATTACATTTATAATTAATTATACACCATGTATATATATGTATTTATATACTATA'

Gli oggetti `AlignedSegment` hanno una serie di metodi:

- `get_tags()` restituisce i tag opzionali

In [61]:
first_alignment.get_tags()

[('NH', 1), ('HI', 1), ('AS', 98), ('nM', 0)]

## Metodi di `AlignmentFile`

- `find_introns()` trova gli introni supportati dagli allineamenti passati come argomento. Per ogni introne, viene fornito il numero di reads allineati (spliced alignment).
       
Viene restituito un oggetto `Counter` in cui le chiavi sono tuple *(start, end)* (inroni) e i valori sono il numero di reads che supportano un dato introne.

Il valore di *end* è la posizione della base del reference immediatamente successiva alla fine dell'introne.

**Esercizio**: estrarre gli introni supportati dagli allineamenti.

In [62]:
bam_file.find_introns(bam_file.fetch())

Counter({(286891, 286948): 1255,
         (287041, 287328): 794,
         (287041, 289040): 468,
         (287526, 289040): 707,
         (289272, 289870): 530,
         (289272, 290252): 730,
         (289943, 290252): 593,
         (290596, 294712): 741,
         (290596, 294822): 586})

- `pileup()`, restituisce un iteratore contenente un oggetto `PileupColumn` per ognuna delle basi coperte da allineamenti che cadono nella regione specificata come *argomento*.

Se l'argomento non viene specificato allora vengono considerati tutti gli allineamenti presenti nel BAM file.

       bam_file.pileup()
       bam_file.pileup(ref_name)
       bam_file.pileup(ref_name, start_region, end_region)

Ogni base della reference coperta da qualche allineamento viene chiamata *pileup column*.

---

Trovare tutte le basi della reference coperte da allineamenti.

In [63]:
pileup_iter = bam_file.pileup()

In [64]:
pileup_columns = list(pileup_iter)

Numero di basi coperte da allineamenti:

In [65]:
len(pileup_columns)

11777

**Esercizio**: accedere alla 1000-esima colonna (base) di *pileup* e:

- settare la qualità minima a 0

In [66]:
pileup_columns[999].set_min_base_quality(0)

- accedere alla posizione sulla reference

In [67]:
pileup_columns[999].pos

284184

- estrarre il numero di reads che coprono la base.

In [68]:
pileup_columns[999].nsegments

180

- ottenere gli identificatori dei reads che coprono la base.

In [69]:
pileup_columns[999].get_query_names()

['FBtr0342963_e_4202_X_294862',
 'FBtr0342963_e_6192_X_294862',
 'FBtr0342963_e_7396_X_294862',
 'FBtr0070103_e_4896_X_294862',
 'FBtr0342963_e_1514_X_294862',
 'FBtr0342963_e_4365_X_294862',
 'FBtr0070103_e_1858_X_294862',
 'FBtr0070103_e_2397_X_294862',
 'FBtr0070103_e_5063_X_294862',
 'FBtr0300326_e_4543_X_294861',
 'FBtr0070103_e_519_X_294861',
 'FBtr0070103_e_7416_X_294861',
 'FBtr0342963_e_2335_X_294861',
 'FBtr0342963_e_4019_X_294861',
 'FBtr0342963_e_4425_X_294861',
 'FBtr0342963_e_4688_X_294861',
 'FBtr0300326_e_2872_X_294861',
 'FBtr0070103_e_1324_X_294861',
 'FBtr0070103_e_4777_X_294861',
 'FBtr0070103_e_6318_X_294861',
 'FBtr0070103_e_8286_X_294861',
 'FBtr0342963_e_4202_X_294862',
 'FBtr0342963_e_6192_X_294862',
 'FBtr0342963_e_7396_X_294862',
 'FBtr0070103_e_4896_X_294862',
 'FBtr0342963_e_1514_X_294862',
 'FBtr0342963_e_4365_X_294862',
 'FBtr0070103_e_1858_X_294862',
 'FBtr0070103_e_2397_X_294862',
 'FBtr0070103_e_5063_X_294862',
 'FBtr0342963_e_4688_X_294861',
 'FBtr030

- estrarre la lista dei reads che coprono la base.

In [70]:
pileup_columns[999].pileups

Per il primo oggetto `PileupRead`, accedere alla posizione sul read che corrisponde alla base di pileup sulla *reference*.

In [71]:
pileup_columns[999].pileups[0].query_position

99

L'allineamento del primo read "impilato" è:

In [72]:
pileup_columns[999].pileups[0].alignment

Recuperare la sua cigarstring.

In [73]:
pileup_columns[999].pileups[0].alignment.cigarstring

'100M'

## Salvare allineamenti in un file `SAM/BAM`

Le seguenti istruzione:

    output_file = pysam.AlignmentFile(output_file_name, 'w', template=template_file)
    output_file = pysam.AlignmentFile(output_file_name, 'wb', template=template_file)
    
aprono un *handle* in scrittura a un file `SAM/BAM` e usa come Header Section quella presente in `template_file`.

Per salvare un allineamento (oggetto `AlignedSegment`), basta scrivere:

    output_file.write(aligned_segment)
    
---

**Esercizio**: trasformare il file BAM in un file SAM.

In [74]:
output_file = pysam.AlignmentFile('./sample.sam', 'w', template=bam_file)

for alignment in all_alignments:
    output_file.write(alignment)

output_file.close()